In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import requests
from datetime import datetime, timedelta
#from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options
import pandas as pd



In [2]:
# moldando data e hora para usar posteriormente
data_hora_atual = datetime.now()
# Formatar data no padrão português brasileiro
data_formatada = data_hora_atual.strftime("%d-%m-%Y")
data_arquivo = data_hora_atual.strftime("%Y_%m_%d")
# Formatar hora no formato hh:mm
hora_formatada = data_hora_atual.strftime("%H_%M_%S")
ano = data_hora_atual.strftime("%Y")
ontem = data_hora_atual - timedelta(days=1)
ontem = ontem.strftime("%d-%m-%Y") # data de ontem formatada para substituir a palavra "ontem" pela data de ontem
# Imprimir data e hora formatadas
#print(f"Data: {data_formatada}")
#print(f"Hora: {hora_formatada}")
#print(f"Ontem foi {ontem}") 

In [3]:
# cabeçalho da requisição para burlar a proteção contra scrap do site do google
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}

# abre o Chrome
#navegador = webdriver.Chrome()
# Configurando o modo headless
options = Options()
options.add_argument("--headless")

# Criando o WebDriver
navegador = webdriver.Chrome(options=options)


In [180]:
######################################### AQUI ACONTECE A INSERÇÃO DO LINK #########################################

link = input("entre com o link da partida")
#link = 'https://www.google.com.br/search?q=rodada+1+premier+league+&sca_esv=72aaa3890415b76e&sca_upv=1&sxsrf=ADLYWIJ6x2PJn90SWsYPqibiPYzcVX4HKg%3A1726583621922&ei=RZPpZqeEON7a5OUPso_okA4&ved=0ahUKEwjnutabmcqIAxVeLbkGHbIHGuIQ4dUDCA8&uact=5&oq=rodada+1+premier+league+&gs_lp=Egxnd3Mtd2l6LXNlcnAiGHJvZGFkYSAxIHByZW1pZXIgbGVhZ3VlIDIFEAAYgAQyBBAAGB4yCBAAGIAEGKIEMggQABiABBiiBDIIEAAYgAQYogRItXZQlQ5Y_3BwA3gBkAEBmAHtAaABohiqAQYwLjE3LjK4AQPIAQD4AQGYAhWgApUXwgIKEAAYsAMY1gQYR8ICChAjGIAEGCcYigXCAgQQIxgnwgIMEAAYgAQYQxiKBRgKwgIKEAAYgAQYQxiKBcICCxAuGIAEGLEDGIMBwgIIEC4YgAQYsQPCAgoQLhiABBhDGIoFwgIQEAAYgAQYsQMYQxiDARiKBcICCxAAGIAEGLEDGIMBwgILEC4YgAQYxwEYrwHCAg0QLhiABBhDGNQCGIoFwgIOEAAYgAQYsQMYgwEYigXCAg4QLhiABBixAxiDARiKBcICCBAAGIAEGLEDwgINEAAYgAQYsQMYQxiKBcICBxAAGIAEGArCAggQABiABBjLAcICBhAAGBYYHsICBxAAGIAEGA3CAgYQABgHGB7CAggQABgHGAgYHpgDAIgGAZAGCJIHBjMuMTcuMaAHnKAB&sclient=gws-wiz-serp#sie=m;/g/11y5mfh0jt;2;/m/02_tc;dt;fp;1;;;'

# acessa o site da partida
navegador.get(link)

In [181]:
# aguarda até a página carregar - aguarda 1 segudno enquanto a quantidade de elementos procurados for zero
while len(navegador.find_elements(By.XPATH, '//*[@id="liveresults-sports-immersive__match-fullpage"]/div/div[2]/div[4]')) < 1:
        sleep(1)
# após encontrar o elemento ele é atribuido a variável tabela
tabela = navegador.find_element(By.XPATH, '//*[@id="liveresults-sports-immersive__match-fullpage"]/div/div[2]/div[4]').get_attribute("outerHTML")

In [182]:
# the site is on de table
##print(tabela)

In [183]:
# a tabela encontrada é "passada a limpo" com essa linha
tabela = BeautifulSoup(tabela, "html.parser")


In [184]:
while len(navegador.find_elements(By.XPATH, '//*[@id="match-stats"]/div/div[1]')) < 1:
    sleep(1)
numeros = navegador.find_element(By.XPATH, '//*[@id="match-stats"]/div/div[1]').get_attribute("outerHTML")


In [185]:
#print(numeros)

In [186]:
numeros =BeautifulSoup(numeros, "html.parser")

## Filtrando o texto dos elementos

In [187]:
# data
data = tabela.find("div", class_="imso-hide-overflow")
data = data.get_text()
#print(data)

## Tratando o texto da variável "data"
> quando pegamos os dados no mesmo dia do jogo a data vem como "Hoje"

In [188]:
# substituindo as palavras "ontem" e "hoje" pelas por datas
if "Hoje" in data:
    data = data_formatada
elif "Ontem" in data:
    data = ontem
else:
    data = f"{data[-10:]}/{ano}"
    data = data.replace('/', '-')
    data = data[-10:]
#print(data)

In [189]:
# nome do mandante do jogo
#mandante = tabela.find("div", class_="imso-hide-overflow")
mandante = tabela.find("div", class_="liveresults-sports-immersive__hide-element")
mandante = mandante.get_text()
#print(mandante)

In [190]:
visitante = tabela.find_all("div", class_="liveresults-sports-immersive__hide-element")

visitante = visitante[1].get_text()
#print(visitante)

In [191]:
gols_mandante = tabela.find("div", class_="imso_mh__l-tm-sc imso_mh__scr-it imso-light-font")
gols_mandante = int(gols_mandante.get_text())
#print(gols_mandante)

In [192]:
gols_visitante = tabela.find("div", class_="imso_mh__r-tm-sc imso_mh__scr-it imso-light-font")
gols_visitante = int(gols_visitante.get_text())
#print(gols_visitante)


In [193]:
escudos = tabela.find_all("img", class_="imso_btl__mh-logo")
escudo_mandante = str(escudos[0])
escudo_visitante = str(escudos[1])
#print(f"{escudo_mandante}\n{escudo_visitante}")


In [194]:
fundamentos = tabela.find_all("tr", class_="MzWkAb")

#chutes = chutes[1].get_text()
##print(chutes)

chutes = fundamentos[0].get_text()
chutes = chutes.split("Chutes")
chutes_mandante = int(chutes[0])
chutes_visitante = int(chutes[1])
#print(f"{mandante} {chutes_mandante} Chutes {chutes_visitante} {visitante}")

chutes_a_gol = fundamentos[1].get_text()
chutes_a_gol = chutes_a_gol.split("Chutes a gol")
chutes_a_gol_mandante = int(chutes_a_gol[0])
chutes_a_gol_visitante = int(chutes_a_gol[1])
#print(f"{mandante} {chutes_a_gol_mandante} Chutes a Gol {chutes_a_gol_visitante} {visitante}") 

posse_de_bola = fundamentos[2].get_text()
posse_de_bola = posse_de_bola.split("Posse de bola")
posse_b_mandante = posse_de_bola[0].replace("%", "")
posse_b_visitante = posse_de_bola[1].replace("%", "")
posse_b_mandante = float(posse_b_mandante)
posse_b_mandante = posse_b_mandante / 100
posse_b_visitante = float(posse_b_visitante)
posse_b_visitante = posse_b_visitante / 100
#print(f"{mandante} {posse_b_mandante} Posse de Bola {posse_b_visitante} {visitante}")

# quantidadae de passes
passes = fundamentos[3].get_text()
passes = passes.split("Passes")
passes_mandante = int(passes[0])
passes_visitante = int(passes[1])
#print(f"{mandante} {passes_mandante} Passes {passes_visitante} {visitante}")

# precisão dos passes
precisao_dos_passes = fundamentos[4].get_text()
precisao_dos_passes = precisao_dos_passes.split("Precisão de passe")
precisao_p_mandante = precisao_dos_passes[0].replace("%", "")
precisao_p_visitante = precisao_dos_passes[1].replace("%","")
precisao_p_mandante = float(precisao_p_mandante)
precisao_p_mandante = precisao_p_mandante / 100
precisao_p_visitante = float(precisao_p_visitante)
precisao_p_visitante = precisao_p_visitante / 100
#print(f"{mandante} {precisao_p_mandante} Precisão Passes {precisao_p_visitante} {visitante}")

# faltas
faltas = fundamentos[5].get_text()
faltas = faltas.split("Faltas")
faltas_mandante = int(faltas[0])
faltas_visitante = int(faltas[1])
#print(f"{mandante} {faltas_mandante} Faltas {faltas_visitante} {visitante}")

# cartões amarelos
cartoes_amarelos = fundamentos[6].get_text()
cartoes_amarelos = cartoes_amarelos.split("Cartões amarelos")
cartoes_a_mandante = int(cartoes_amarelos[0])
cartoes_a_visitante = int(cartoes_amarelos[1])
#print(f"{mandante} {cartoes_a_mandante} Cartões Amarelos {cartoes_a_visitante} {visitante}")

# cartõese vermelhos
cartoes_vermelhos = fundamentos[7].get_text()
cartoes_vermelhos = cartoes_vermelhos.split("Cartões vermelhos")
cartoes_v_mandante = int(cartoes_vermelhos[0])
cartoes_v_visitante = int(cartoes_vermelhos[1])
#print(f"{mandante} {cartoes_v_mandante} Cartões Vermelhos {cartoes_v_visitante} {visitante}")

# impedimentos
impedimentos = fundamentos[8].get_text()
impedimentos = impedimentos.split("Impedimentos")
impedimentos_mandante = int(impedimentos[0])
impedimentos_visitante = int(impedimentos[1])
#print(f"{mandante} {impedimentos_mandante} Impedimentos {impedimentos_visitante} {visitante}")

# escanteios
escanteios = fundamentos[9].get_text()
escanteios = escanteios.split("Escanteios")
escanteios_mandante = int(escanteios[0])
escanteios_visitante = int(escanteios[1])
#print(f"tivemos {escanteios_mandante + escanteios_visitante} no jogo, sendo {escanteios_mandante} do {mandante} e {escanteios_visitante} do {visitante}")


In [195]:

if gols_mandante > gols_visitante:
    vencedor = mandante
    derrotado = visitante
elif gols_mandante < gols_visitante:
    vencedor = visitante
    derrotado = mandante
else:
    vencedor = '-'
    derrotado = '-'
#print(f"O {vencedor} venceu o {derrotado}")
#print(f"PLACAR: {mandante} {gols_mandante} x {gols_visitante} {visitante}")



In [196]:
# toca um sinal de que o código foi terminado e fecha o navegador
navegador.get(f"https://translate.google.com/?hl=en&tab=TT&sl=auto&tl=en&text={mandante}%20{gols_mandante}%20x%20{gols_visitante}%20{visitante}.%0Anavegador%20permanece%20aberto&op=translate")
while len(navegador.find_elements(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div/div[2]/div[2]/div/c-wiz/div[5]/div/div[2]/div/div[2]/span/button/div[3]')) < 1:
    sleep(1) # aguarda a página carregar
navegador.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div/div[2]/div[2]/div/c-wiz/div[5]/div/div[2]/div/div[2]/span/button/div[3]').click() # clica no botão de aceitar os cookies
sleep(5) # aguarda o áudio tocar completamente


In [197]:
#navegador.quit() # fecha a bodega

In [198]:
dados_jogo = {
    "data": data,
    "img mand": escudo_mandante,
    "img visit": escudo_visitante,
    "mandante": mandante,
    "visitante": visitante,
    "gols mand.": gols_mandante,
    "gols_visit.": gols_visitante,
    "chutes mandante": chutes_mandante,
    "chutes visitante": chutes_visitante,
    "chutes a gols mand.": chutes_a_gol_mandante,
    "chutes a gols visit.": chutes_a_gol_visitante,
    "posse de b mand.": posse_b_mandante,
    "posse b visit..": posse_b_visitante,
    "passes mand.": passes_mandante,
    "passes visit.": passes_visitante,
    "precisao p mand.": precisao_p_mandante,
    "precisao p visit.": precisao_p_visitante,
    "faltas mand.": faltas_mandante,
    "faltas visit.": faltas_visitante,
    "cart. amar. mand.": cartoes_a_mandante,
    "cart. amar. visit.": cartoes_a_visitante,
    "cart verm. mand.": cartoes_v_mandante,
    "cart vaerm. visit.": cartoes_v_visitante,
    "impedimento mand.": impedimentos_mandante,
    "impedimento visit.": impedimentos_visitante,
    "escanteios mand.": escanteios_mandante,
    "escanteios visit.": escanteios_visitante


}

colunas = [data, escudo_mandante, escudo_visitante, mandante, visitante, gols_mandante, gols_visitante, vencedor, derrotado, chutes_mandante, chutes_visitante, chutes_a_gol_mandante, chutes_a_gol_visitante, posse_b_mandante, posse_b_visitante, passes_mandante, passes_visitante, precisao_p_mandante, precisao_p_visitante, faltas_mandante, faltas_visitante, cartoes_a_mandante, cartoes_a_visitante, cartoes_v_mandante, cartoes_v_visitante, impedimentos_mandante, impedimentos_visitante, escanteios_mandante, escanteios_visitante]
df = pd.DataFrame(columns=colunas, index=["0"])
df

,15-09-2024,"<img alt="""" class=""imso_btl__mh-logo"" height=""48px"" id=""spotl_JefqZuLaIKbK1sQP2KfgqQ8_1"" src=""//ssl.gstatic.com/onebox/media/sports/logos/orE554NToSkH6nuwofe7Yg_96x96.png"" width=""48px""/>","<img alt="""" class=""imso_btl__mh-logo"" height=""48px"" id=""spotl_JefqZuLaIKbK1sQP2KfgqQ8_3"" src=""//ssl.gstatic.com/onebox/media/sports/logos/hHwT8LwRmYCAGxQ-STLxYA_96x96.png"" width=""48px""/>",Flamengo,Vasco da Gama,1,1,-,-,17,...,10,10,0,1,0,0,3,1,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
# a tabela com os dados do jogo é expressa por esses dados
#print(numeros)

<div class="lr-imso-ss-bb lr-imso-ss-wdm"><table class="lr-imso-ss-wdt"><tbody><tr class="zvQp2c"><th aria-label="Wolves" class="jqZdce" role="columnheader" scope="col"><div aria-hidden="true"><div class="UNCnvc"><img alt="Wolves" class="imso_btl__mh-logo" data-ilt="1726576456449" height="24px" id="spotl_RHfpZtOCFtXR1sQPwuPT2Qg_9" src="//ssl.gstatic.com/onebox/media/sports/logos/-WjHLbBIQO9xE2e2MW3OPQ_48x48.png" width="24px"/></div></div></th><th class="gZIgRd imso-medium-font" role="columnheader" scope="col">ESTATÍSTICAS DOS TIMES</th><th aria-label="Newcastle" class="jqZdce" role="columnheader" scope="col"><div aria-hidden="true"><div class="UNCnvc"><img alt="Newcastle" class="imso_btl__mh-logo" data-ilt="1726576456449" height="24px" id="spotl_RHfpZtOCFtXR1sQPwuPT2Qg_11" src="//ssl.gstatic.com/onebox/media/sports/logos/96CcNNQ0AYDAbssP0V9LuQ_48x48.png" width="24px"/></div></div></th></tr><tr class="MzWkAb"><td>12</td><th class="JmSkkf" role="rowheader" scope="row">Chutes</th><td>14</td></tr><tr class="MzWkAb"><td>5</td><th class="JmSkkf" role="rowheader" scope="row">Chutes a gol</th><td>6</td></tr><tr class="MzWkAb"><td>48%</td><th class="JmSkkf" role="rowheader" scope="row">Posse de bola</th><td>52%</td></tr><tr class="MzWkAb"><td>433</td><th class="JmSkkf" role="rowheader" scope="row">Passes</th><td>471</td></tr><tr class="MzWkAb"><td>83%</td><th class="JmSkkf" role="rowheader" scope="row">Precisão de passe</th><td>88%</td></tr><tr class="MzWkAb"><td>17</td><th class="JmSkkf" role="rowheader" scope="row">Faltas</th><td>6</td></tr><tr class="MzWkAb"><td>3</td><th class="JmSkkf" role="rowheader" scope="row">Cartões amarelos</th><td>3</td></tr><tr class="MzWkAb"><td>0</td><th class="JmSkkf" role="rowheader" scope="row">Cartões vermelhos</th><td>0</td></tr><tr class="MzWkAb"><td>3</td><th class="JmSkkf" role="rowheader" scope="row">Impedimentos</th><td>2</td></tr><tr class="MzWkAb"><td>4</td><th class="JmSkkf" role="rowheader" scope="row">Escanteios</th><td>7</td></tr></tbody></table></div>

In [200]:
# salvando em arquivo - padrãao: "[data]_-_[placar]_-_[hora de criação do arquivo].extensão"
df.to_excel(f"{data}_-_{mandante}_{gols_mandante}_x_{gols_visitante}_{visitante}_-_{hora_formatada}.xlsx")

In [201]:
cabecalho = [
    data, escudo_mandante, escudo_visitante, mandante, visitante, gols_mandante, gols_visitante, vencedor, derrotado, chutes_mandante, chutes_visitante, chutes_a_gol_mandante, chutes_a_gol_visitante, posse_b_mandante, posse_b_visitante, passes_mandante, passes_visitante, precisao_p_mandante, precisao_p_visitante, faltas_mandante, faltas_visitante, cartoes_a_mandante, cartoes_a_visitante, cartoes_v_mandante, cartoes_v_visitante, impedimentos_mandante, impedimentos_visitante, escanteios_mandante, escanteios_visitante

]

#print(cabecalho)